### import, functions

In [1]:
import pyperclip
import win32gui, win32con
import time,re, os
import pandas as pd
import pyautogui as au
# import re
# import os
# import numpy as np
# from tqdm import tqdm
from os.path import join
import random
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.select import Select
from selenium.webdriver.common.alert import Alert
# from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoAlertPresentException #, NoSuchWindowException # NoSuchElementException, UnexpectedAlertPresentException, WebDriverException,
# from file_function import read_filelist
from datetime import datetime
# from datetime import timedelta
import traceback
# 정렬순서 : 확정일자(내림), 법원(올림), 사건번호(내림)

dtype = {'채무자키':str, '보증인키':str, '가상계좌번호':str, '법조치키':str}

#############################
보조pc = True
#############################

if 보조pc : 
    def lwt() : time.sleep(2.8)
    def mwt() : time.sleep(1.2)
    def swt() : time.sleep(random.uniform(0.3,0.6))
else : 
    def lwt() : time.sleep(2.5)
    def mwt() : time.sleep(1)
    def swt() : time.sleep(random.uniform(0.2,0.5))
    

def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 감지 방지
    chrome_options.add_experimental_option("useAutomationExtension", False)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

def con_scourt() :
    driver = set_chrome_driver()
    driver.get("https://edi.ccrs.or.kr/fss/index.jsp")
    
    driver.maximize_window() # 최대화 필수(상단 메뉴 보였다 안 보였다...)
    wait = WebDriverWait(driver, 7) 
    return driver, wait

def get_open_windows():
    """현재 열려있는 창목록을 튜플로 반환. print가능"""
    windows = []
    def callback(hwnd, extra):
        if win32gui.IsWindowVisible(hwnd):  # 보이는 창만 가져오기
            title = win32gui.GetWindowText(hwnd)
            if title:
                windows.append((hwnd, title))

    win32gui.EnumWindows(callback, None)
    return windows
        
def wait_for_window_to_close(window_title, timeout=15):
    """
    특정 창이 나타났다가 사라질 때까지 대기하는 함수
    :param window_title: 감시할 창의 제목 (부분 일치 가능)
    :param timeout: 최대 대기 시간 (초)
    """
    start_time = time.time()
    cnt = 0 # 최초 감지하면 1로
    while time.time() - start_time < timeout:
        hwnd = win32gui.FindWindow(None, window_title)
        
        if hwnd:  # 창이 존재하면 계속 확인
            if cnt ==0 : #최초 감지시
                print(f"'{window_title}' 창 감지됨. 닫힐 때까지 대기 중...")
                cnt += 1
            else : pass
        else:  # 창이 사라지면 종료
            if cnt == 0 : # 아직 수작업 안 끝남
                pass
            else :
                print(f"'{window_title}' 창이 닫혔습니다. 다음 단계 진행.")
                return True

        time.sleep(1)  # 1초마다 확인
    
    print(f"타임아웃: '{window_title}' 창이 {timeout}초 내에 닫히지 않았습니다.")
    return False
       

def 키워드로파일명찾기(폴더:str, 포함키워드:str, 제외키워드="", 전체경로=True, 여러파일허용=False) :
    file_list = [file for file in os.listdir(폴더) if not file.startswith('~$')]
    if 제외키워드 == "" : 
        fn = [file for file in file_list if re.search(포함키워드, file)]
    else : 
        fn = [file for file in file_list if (re.search(포함키워드, file)!=None) & (re.search(제외키워드,file)==None)]
    
    if len(fn) == 1 :
        if 전체경로 : 
            return join(폴더,fn[0])
        else : 
            return fn[0]
    elif len(fn) == 0 :
        print(f"포함키워드:{포함키워드}, 제외키워드{제외키워드} 조건을 만족하는 파일이 없습니다.")
    else : 
        if 여러파일허용 : 
            if 전체경로 : 
                return [join(폴더,f) for f in fn]
            else : 
                return fn
        else : 
            print(f"포함키워드:{포함키워드}, 제외키워드{제외키워드} 조건을 만족하는 파일이 둘 이상입니다.")
            print(fn)

def close_all_popus(driver) :
    try:
        # 모든 팝업 창 닫기
        if len(driver.window_handles) > 1 : # 메인브라우저가 언제나 0번으로 보이고, 팝업은 1~, 새로고침해도 브라우저 키값은 변하지 않음
            for handle in driver.window_handles[1:]:
                driver.switch_to.window(handle)
                driver.close()
                swt()
        
        # 첫 번째 창으로 돌아가기
        driver.switch_to.window(driver.window_handles[0])
        swt()
        
    except Exception as e:
        print(f"팝업 창을 닫는 동안 오류가 발생했습니다: {e}")

def driver_refresh(driver):
    driver.switch_to.window(driver.window_handles[0])
    swt()
    # 브라우저 새로고침
    driver.refresh()
    lwt()
    lwt()
    
        
def 고객현황메뉴클릭(driver, wait) : 
    왼쪽날개메뉴id = 'mainframe.VFrameSet.HFrameSet.frameLeft.form.grdMenu.body.gridrow_1.cell_1_0.celltreeitem.treeitemtext:text'
    주민등록번호css = "div[id^='mainframe'][id$='radioitem2:icontext']"
    # 고객 현황조회 클릭
    wait.until(EC.element_to_be_clickable((By.ID, 왼쪽날개메뉴id))).click()
    mwt()
    # 주민등록번호 선택
    radio_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 주민등록번호css)))
    radio_button.click()
    mwt()

def value_clear_and_sendkeys(ele, keys) :
    ele.click()
    if ele.get_attribute("value"):  # 값이 존재하면
        ele.clear()  # 입력 필드 지우기
        swt()
    swt()
    ele.send_keys(keys)
    swt()

def login(driver, id, pw) :
    id_id = "mainframe.VFrameSet.LoginFrame.form.divLogin.form.edtId:input"
    pw_id = "mainframe.VFrameSet.LoginFrame.form.divLogin.form.edtPw:input"
    id_ele = driver.find_element(By.ID, id_id)

    value_clear_and_sendkeys(id_ele, id)

    pw_ele = driver.find_element(By.ID, pw_id)
    value_clear_and_sendkeys(pw_ele, pw)

    driver.find_element(By.ID, "mainframe.VFrameSet.LoginFrame.form.divLogin.form.btnLogin").click()
    lwt()

# 창 찾기
# 타이틀로 : win32gui.FindWindow(None, "창이름")
# 창번호로(재실행시 변경됨) : win32gui.IsWindow(657416)

### 파일 읽기 및 변수지정

In [2]:
#######################################
id = 'help02'
pw = '***removed****'
#######################################

In [13]:
# 파일 읽기  ############### 가장 최근 파일 읽는 거로 자동변경하자

fn_base = "result_서치용목록_파일명추가"
if 보조pc :
    path_base = r"\\192.168.10.83\Users\DATA\Desktop\신용회복 파일 다운"
else :
    path_base = r"C:\Users\DATA\Desktop\신용회복 파일 다운"
fn_read_list = 키워드로파일명찾기(path_base,fn_base,여러파일허용=True)
if isinstance(fn_read_list, list) :
    fn_read_list.sort(reverse=True)
    fn_read = fn_read_list[0] # 최근 파일
else :
    fn_read = fn_read_list
print(f'파일 읽기 : {fn_read}')
df_ori = pd.read_excel(fn_read, dtype=dtype)

파일 읽기 : \\192.168.10.83\Users\DATA\Desktop\신용회복 파일 다운\result_서치용목록_파일명추가_250227_133536.xlsx


In [14]:
##########################################
채권사 = "솔림"
suffix = "_고객" # _최초조정, _재조정, _원상 (출력할 파일 종류)
출력창이름 ="신청인현황"
##########################################
저장창이름 = '다음 이름으로 프린터 출력 저장'
중복저장 = '다른 이름으로 저장 확인'

# 컬럼명 stem : 서버건수, 서버파일명, 파일명, 출력여부
df = df_ori[(df_ori["채권사"]==채권사)&(df_ori[f"서버건수{suffix}"]==0)&(df_ori[f"출력여부{suffix}"].isna())]
print(f'총작업건수 : {len(df)}')
if len(df) > 0 : print(df.iloc[0][["파일명"+suffix,'주민번호']])

총작업건수 : 0


### 실행

In [ ]:
# 건당 30초남짓, 255건 연속 출력하면 뷰어설치 에러 나는듯
##########################################
새로고침번호 = 200
epoch = 2 # 시간당 약 218건
##########################################
작업할당량 = 새로고침번호 * epoch
epoch_num = 0
epoch_start_time = datetime.now()
while(True) :
    epoch_num+=1
    # 작업목록 재작성
    df = df_ori[(df_ori["채권사"]==채권사)&(df_ori[f"서버건수{suffix}"]==0)&(df_ori[f"출력여부{suffix}"].isna())]
    
    # 작업할 거 없으면 while 문 종료
    if len(df) == 0 :
        print("+++++++++++++++++++++++++모든 목록을 완료하여 작업을 종료합니다.+++++++++++++++++++++++++")
        formatted_time = time.strftime("%y%m%d_%H%M%S", time.localtime(time.time()))
        formatted_time
        df_ori.to_excel(join(path_base, fn_base+"_"+formatted_time+".xlsx"), index=False)
        print('모든 작업완료, 파일저장')
        break

    print(f'총작업할 건수 : {len(df)}')
    print("최초작업건",df.iloc[0]["파일명"+suffix])
    
    # 드라이버 열고 반복작업준비
    driver, wait = con_scourt() # 로그인 ~ 주민번호선택까지는 수동
    time.sleep(5)

    login(driver, id, pw)

    close_all_popus(driver)

    # 브라우저와 vscode 번호 확인하기
    wnds = get_open_windows()
    print(wnds[:3])
    vscode,browser = 0,0
    for i, v in wnds :
        if re.search("sele_신복파일.ipynb", v) :
            vscode = i
        elif re.search("채권금융회사 업무시스템.*Chrome", v) :
            browser = i
        if (vscode!=0) and (browser!=0) : 
            break
    print(f'vscode_win_num : {vscode},brower_win_num : {browser}')
    if not(vscode and browser) :
        print("창 코드번호를 읽지못함")
        break
    

    # 새로고침 및 조회화면까지
    고객현황메뉴클릭(driver, wait) 
    swt()

    # 반복문 ----------------------------------------------------------------------
    # cnt 초기화
    cnt = 0
    try :
        for i, v in df.iloc[:작업할당량].iterrows() : 
            파일명 = v[f'파일명{suffix}']
            
            # 브라우저 활성화
            # win32gui.ShowWindow(browser, win32con.SW_RESTORE)  # 창이 최소화되었을 경우 복구
            # time.sleep(0.1)  # 약간의 지연을 줘서 시스템이 따라갈 수 있도록 함
            # if cnt == 0 :pass
            # else : win32gui.SetForegroundWindow(browser)
            swt()

            # 식별번호 입력
            goon_num = 0
            while(goon_num<3) : # 3회 반복
                ele = driver.find_element(By.CSS_SELECTOR, 'div[id^="mainframe"][id$="form.divSearch.form.maeSsno"] input')
                if ele.get_attribute("value"):  # 값이 존재하면
                    ele.clear()  # 입력 필드 지우기
                    swt()
                
                ele.click()
                swt()
                ele.send_keys(v.주민번호)
                
                if ele.get_attribute("value") == v.주민번호 :
                    break
                else :
                    goon_num += 1

            swt()
            # 조회버튼 클릭
            driver.find_element(By.CSS_SELECTOR, 'div[id^="mainframe"][id$="form.divWork.form.divSearch.form.btnSelect:icontext"] div').click()
            mwt()
                
            try : # 오류 : 대상자 없음
                alert = driver.switch_to.alert
                Alert(driver).accept() # 오류창 확인버튼 클릭
                swt()
                df_ori.loc[i,f"출력여부{suffix}"] = "정보없음"
                continue
            except NoAlertPresentException : # 정상
                pass
            
            # 출력버튼 클릭
            ele = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[id^="mainframe"][id$="form.divWork.form.divPrint_01.form.btnPrint_01"] div')))
            # ele = driver.find_element(By.CSS_SELECTOR, 'div[id^="mainframe"][id$="form.divWork.form.divPrint_01.form.btnPrint_01"] div')
            ele.click()
            lwt()

            # 문서확인 및 마우스 이동
            while(True) :
                hwnd = win32gui.FindWindow(None, 출력창이름)
                if hwnd :
                    win32gui.SetForegroundWindow(hwnd)
                    break
                else : 
                    time.sleep(1)

            # 출력명령
            mwt()
            au.hotkey('ctrl','p')
            
            # 인쇄 확인
            lwt()           
            au.press('enter')

            # 파일명 복사
            pyperclip.copy(파일명)
            swt()
                # 저장창 활성화
            win32gui.SetForegroundWindow(win32gui.FindWindow(None, 저장창이름)) 
            swt()
                # 붙여넣기
            au.hotkey('ctrl','v')
            swt()
            au.press('enter') ######### 구폴더작업 폴더에 저장하자
            mwt()
            
            # 파일명 중복 여부 체크
            중복창num = win32gui.FindWindow(None, 중복저장) # 이미 파일 있는 경우
            if  중복창num : # 오류
                print('중복파일 발생!!! 처리 후 코드 재진행 원할시 0')
                print('출력여부 수정할 거는 따로 메모해둬라')
                win32gui.SetForegroundWindow(vscode) # vscode 활성화
                goon = input()
            
                # input창으로 마우스 클릭
                if goon == "0" :
                    continue
                else :
                    break
            else : # 정상
                df_ori.loc[i,f"출력여부{suffix}"] = "출력"
                lwt() #저장하는 동안 기다리기
            cnt += 1

            # 새로고침번호 도달시 for문 벗어나서 브라우저 새로고침
            if cnt==새로고침번호 :
                # 작업할당량 재설정 후 for문 벗어나기
                print(f"-----------------------epoch {epoch_num}회, 총 {새로고침번호*epoch_num}건 출력({새로고침번호}*{epoch_num}) ---------------------------")
                작업할당량 -= cnt
                # 출력창닫기 (팝업 아니므로 close_all_popus()안 됨)
                hwnd = win32gui.FindWindow(None, 출력창이름)
                if hwnd :
                    win32gui.SetForegroundWindow(hwnd)
                    swt()
                    win32gui.PostMessage(hwnd, win32con.WM_CLOSE, 0, 0)

                mwt()
                driver.close() # 드라이버 종료
                time.sleep(30)
                break
    except :
        epoch_end_time = datetime.now()
        print("마지막(재시작 할) 작업파일 :", i, 파일명,"****************")
        print(f"총 출력건수 : {새로고침번호*(epoch_num-1)+cnt}","###################")
        print(f'시작시간 : {epoch_start_time}, 종료시간:{epoch_end_time}, 소요시간 : {epoch_end_time-epoch_start_time}')
        print('------------------------------------------------')
        print(traceback.format_exc())

        formatted_time = time.strftime("%y%m%d_%H%M%S", time.localtime(time.time()))
        formatted_time
        df_ori.to_excel(join(path_base, fn_base+"_"+formatted_time+".xlsx"), index=False)
        print('파일 저장 완료')
        # 출력창닫기 (팝업 아니므로 close_all_popus()안 됨)
        hwnd = win32gui.FindWindow(None, 출력창이름)
        if hwnd :
            win32gui.SetForegroundWindow(hwnd)
            swt()
            win32gui.PostMessage(hwnd, win32con.WM_CLOSE, 0, 0)
        mwt()
        # driver.close() # 드라이버 종료
        break # 예외 발생 시 while문까지 종료


총작업할 건수 : 340
최초작업건 20479983 박혜선 고객현황 2021-08-27
[(3870010, '운영 :: 채권금융회사 업무시스템 - 채권금융회사 로그인 - Chrome'), (66824, '● sele_신복파일.ipynb - fileNaming - Visual Studio Code [Administrator]'), (131664, '설정')]
vscode_win_num : 66824,brower_win_num : 3870010
-----------------------epoch 1회, 총 200건 출력(200*1) ---------------------------
총작업할 건수 : 125
최초작업건 20444187 윤상례 고객현황 2022-01-06
[(35393032, '운영 :: 채권금융회사 업무시스템 - 채권금융회사 로그인 - Chrome'), (66824, '● sele_신복파일.ipynb - fileNaming - Visual Studio Code [Administrator]'), (131664, '설정')]
vscode_win_num : 66824,brower_win_num : 35393032
+++++++++++++++++++++++++모든 목록을 완료하여 작업을 종료합니다.+++++++++++++++++++++++++


In [6]:

    # ctrl p 안 먹었을 때 코드 ############    
    # # 다른 이름으로 프린터 출력 저장 사라졌는지
    # 저장완료여부 = wait_for_window_to_close(저장창이름)
    
    # if 저장완료여부 :
    #     time.sleep(2) # 저장하는 동안 쉬어야 input창 활성화
    #     df_ori.loc[i,"출력여부"] = "출력"
    #     win32gui.SetForegroundWindow(vscode) # vscode 활성화
    #     goon = input()
    
    #     # input창으로 마우스 클릭
    #     if goon == "0" :
    #         continue
    #     else :
    #         break
